# IMPORT CSV INTO ORACLE DB 

CONNECTION TO ORACLE AND IMPORT REQUIRE LIBRARIES

In [ ]:
import pandas as pd
import glob
import os
import cx_Oracle
from cx_Oracle import DatabaseError
dsn_tns = cx_Oracle.makedsn(' Hostname ', ' Port ', service_name=' Service Name ')
con = cx_Oracle.connect(user='Username', password='Password', dsn=dsn_tns)

print("conected to oracle db")


cursor = conn.cursor()

cursor.execute("""ALTER SESSION SET NLS_DATE_FORMAT = 'DD-MM-YYYY HH24:MI:SS'
                                    NLS_LANGUAGE = AMERICAN""")

print('You are connected: ')



CREATE TABLE IN ORACLE DB

In [ ]:
# q="""CREATE TABLE XTEMP_LOAD_UNITS
#            (   UNIT_LICENCE_PLATE VARCHAR2(50 CHAR)
#              , MODIFICATION_TIME DATE
#              , PARENT_LP VARCHAR2(50 CHAR)
#              , HANDLING_TYPE_NAME VARCHAR2(20 CHAR)
#              , URN VARCHAR2(50 CHAR))"""
# cursor.execute(q)
# conn.commit()

# print('Temporary table created!!! ')


IMPORT CSV INTO DATAFRAME AND CHANGE DATA TYPES


In [ ]:
FILE PATH = "W:\\Tushar\\Access_db_exports\\_Export_Load_Units_2907.txt"
csv_data = pd.read_csv( FILE PATH ,index_col=False,delimiter= '|')
#csv_data.head()
data = []
csv_data['Modification Time']= pd.to_datetime(csv_data['Modification Time']).dt.strftime('%d-%m-%Y %H:%M:%S')
csv_data = csv_data.astype({'Unit Licence Plate':str, 'Parent LP':str})

print('Inserted data into dataframe....')

In [ ]:
csv_data.head()

INSERT DATAFRAME INTO ORACLE DB TABLE

In [ ]:
print('Inserting data into table....')

batch_size= 16737.5

for i,line in csv_data.iterrows():
    sql="""insert into XTEMP_LOAD_UNITS (   UNIT_LICENCE_PLATE
                                               , MODIFICATION_TIME
                                               , PARENT_LP
                                               , HANDLING_TYPE_NAME
                                               , URN
                                               , TYPE
                                                ) 
            values (:0,:1,:2,:3,:4,:5)"""
    data.append((line[0],line[1],line[2],line[3]))
    if len(data) % batch_size == 0:
        cursor.executemany(sql, data)
        data = []        
    
        # the connection is not autocommitted by default, so we must commit to save our changes
        
#     if data:
#         cursor.executemany(sql, data)
        conn.commit()

print('data loaded into the table successfully!!!') 